# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Direct Preference Optimization 

__Мягкий дедлайн 16.11.25 23:59__ \
__Жесткий дедлайн 19.11.25 23:59__

### О задании

В этом задании вам предстоит обучить большую LLM для ответов на вопросы с помощью DPO, а также реализовать LoRA для эффективного обучения. 

### Оценивание и штрафы

Максимально допустимая оценка за работу — __11 баллов__.

Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета баллы за соответствующие задания не будут ставиться. Мы настаиваем на том, чтобы вы оформили весь код в виде полноценного проекта. Этот ноутбук нужно рассматривать скорее как файл с условием, чем как место для написания массивного кода. За сдачу больших ноутбуков с кодом оценка будет снижена. Ответы на все вопросы в заданиях можно (нужно) писать в отчете.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

### План решения

<img src="https://miro.medium.com/v2/resize:fit:1400/1*lK6iJMz5CGh2fo7TsDn15A.png" alt="drawing" width="700"/>

Обучение следованию инструкциям с помощью DPO разбивается на два этапа:    
1. __Supervised Fine-tuning (SFT)__ – обучение базовой модели ответам на запросы в нужном формате.
2. __Direct Preference Optimization (DPO)__ – обучение SFT модели приоритизации "хороших" ответов.

Мы не хотим обучать модели целиком по двум причинам: 1) используемые модели очень большие; 2) нам требуется лишь выравнить модель с нашими предпочтениями, не внося в нее новых знаний, что не требует серьезного обучения. Поэтому мы будем использовать PEFT, а именно LoRA для обучения.

Таким образом, вам надо будет:
1. Реализовать и протестировать LoRA
2. Разобраться с данными и привести их к нужному формату
3. Обучить SFT модель
4. Обучить DPO модель
5. Порадоваться, что вы молодцы и со всем справились
6. (Опционально) сделать веб-интерфейс для вашей модели, переиспользуя код из первой домашки (мы можем выдать бонусы, если получится классно).

### О датасете

Мы будем работать с датасетом [Anthropic Helpful-Harmless](https://huggingface.co/datasets/Anthropic/hh-rlhf) для RLHF. В нем содержится 160к примеров ответов на вопросы с историей.

### Low-Rank Adaptation (LoRA)

<img src="https://heidloff.net/assets/img/2023/08/lora.png" alt="drawing" width="600"/>

__Задание 1 (3 балла).__ Реализуйте самостоятельно модуль LoRA для эффективного обучения LLM по схеме, описанной в [статье](https://arxiv.org/pdf/2106.09685). Встройте его в свою любимую LLM и убедитесь, что ошибка убывает при обучении параметров LoRA на безусловную генерацию. Для этого возьмите любые данные на свой выбор. Замерьте насколько уменьшилось число обучаемых параметров, как изменилась скорость во время forward и backward процессов и как изменились затраты по памяти. Сделайте выводы и напишите о них в отчете.

In [1]:
import torch

In [2]:
# your code here
class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.A = torch.nn.Parameter(torch.randn(in_dim, rank) * std_dev) #норм распределение с параметрами 0 и сигма квадрат
        self.B = torch.nn.Parameter(torch.zeros(rank, out_dim))

    def forward(self, x):
        x = (x @ self.A) @ self.B
        return x

In [3]:
class LinearWithLoRA(torch.nn.Module):
    def __init__(self, linear, rank):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(linear.in_features, linear.out_features, rank)

        device = linear.weight.device
        dtype = linear.weight.dtype

        self.lora.A = torch.nn.Parameter(self.lora.A.to(device=device, dtype=dtype))
        self.lora.B = torch.nn.Parameter(self.lora.B.to(device=device, dtype=dtype))

    def forward(self, x):
        return self.linear(x) + self.lora(x)


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B")

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [13]:
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [9]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

Total parameters: 1543714304
Trainable parameters: 1543714304


In [4]:
def _replace_module(module):
        for name, child in module.named_children():
            if isinstance(child, torch.nn.Linear):
                lora_layer = LinearWithLoRA(child, rank=8)
                setattr(module, name, lora_layer)
            else:
                _replace_module(child)
                
def replace_with_lora(model, rank=8):    
    _replace_module(model)
    return model

In [ ]:
for param in model.parameters():
    param.requires_grad = False

model = replace_with_lora(model, rank=8)

In [11]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

Total parameters: 1554174464
Trainable parameters: 10460160


In [7]:
from datasets import load_dataset
train_dataset = load_dataset("roneneldan/TinyStories", split="train[:1000]")
test_dataset = load_dataset("roneneldan/TinyStories", split="validation[:100]")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004-2d5a1467fff108(…):   0%|          | 0.00/249M [00:00<?, ?B/s]

data/train-00001-of-00004-5852b56a2bd28f(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/train-00002-of-00004-a26307300439e9(…):   0%|          | 0.00/246M [00:00<?, ?B/s]

data/train-00003-of-00004-d243063613e5a0(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/validation-00000-of-00001-869c898b5(…):   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

In [8]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [8]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"], 
        truncation=True, 
        padding="max_length", 
        max_length=128
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True,remove_columns=train_dataset.column_names)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True,remove_columns=test_dataset.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [17]:
print(next(model.parameters()).device) 

cuda:0


In [18]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

import os
os.environ["WANDB_DISABLED"] = "true"  

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir='./lora_qwen',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    dataloader_pin_memory=True,
    eval_steps=500,
    report_to=None,  
    logging_first_step=True,  
    disable_tqdm=False,  
    remove_unused_columns=False,
    save_safetensors=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator
)


trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
1,0.577200
10,0.605700
20,0.564500
30,0.610900
40,0.603800
50,0.579900
60,0.608900
70,0.648600
80,0.616000
90,0.673300


TrainOutput(global_step=375, training_loss=0.6107760910987854, metrics={'train_runtime': 457.6152, 'train_samples_per_second': 6.556, 'train_steps_per_second': 0.819, 'total_flos': 3043124969472000.0, 'train_loss': 0.6107760910987854, 'epoch': 3.0})

In [ ]:
device

device(type='cuda')

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B",
    torch_dtype=torch.float16,
    device_map=None,  
    trust_remote_code=True
)

base_model.to(device)

model = replace_with_lora(base_model, rank=8)

state_dict = torch.load("/kaggle/input/qwen-lora/transformers/default/1/pytorch_model-00001-of-00002.bin", map_location="cpu")
state_dict2 = torch.load("/kaggle/input/qwen-lora/transformers/default/1/pytorch_model-00002-of-00002.bin", map_location="cpu")
state_dict.update(state_dict2)

model.load_state_dict(state_dict, strict=False)

model.eval()

prompt = "tell me a story about a red hat."

inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=100)

print(tokenizer.decode(output[0], skip_special_tokens=True))

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


tell me a story about a red hat. Once upon a time, there was a little girl named Lily. She loved to play outside in the sunshine. One day, she went to the park to play. As she was walking, she saw a red hat on a bench. It was a big red hat that looked like it belonged to a superhero. Lily was so excited! She ran over to the hat and picked it up. She put it on her head and felt so brave. She ran back to the playground and played with her friends


In [24]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B",
    torch_dtype=torch.float16,
)

In [25]:
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [6]:
import gc

torch.cuda.empty_cache()
gc.collect()

0

In [12]:
import time

def get_batch(dataset, batch_size=2, max_length=16):
    idx = torch.randint(0, len(dataset), (batch_size,)).tolist()
    
    batch = {
        "input_ids": torch.stack([torch.tensor(dataset[i]["input_ids"][:max_length]) for i in idx]).to("cuda"),
        "labels": torch.stack([torch.tensor(dataset[i]["labels"][:max_length]) for i in idx]).to("cuda"),
        "attention_mask": torch.stack([torch.tensor(dataset[i]["attention_mask"][:max_length]) for i in idx]).to("cuda"),
    }
    return batch


def check_time(model, name="", enable_backward=True):
    device = next(model.parameters()).device
    print(f"Model is on device: {device}")

    model.train()
    
    if enable_backward:
        optimizer = torch.optim.AdamW(
            [p for p in model.parameters() if p.requires_grad],
            lr=1e-4
        )
    else:
        optimizer = None

    for _ in range(2):
        batch = get_batch(tokenized_train_dataset, batch_size=1, max_length=32)
        batch = {k: v.to(device) for k, v in batch.items()}
        out = model(**batch)
        loss = out.loss
        if enable_backward:
            optimizer.zero_grad()
            loss.backward()

    torch.cuda.synchronize()

    batch = get_batch(tokenized_train_dataset, batch_size=1, max_length=32)
    batch = {k: v.to(device) for k, v in batch.items()}
    
    start = time.time()
    out = model(**batch)
    torch.cuda.synchronize()
    fwd = time.time() - start

    if enable_backward:
        loss = out.loss
        optimizer.zero_grad()
        start = time.time()
        loss.backward()
        torch.cuda.synchronize()
        bwd = time.time() - start
    else:
        bwd = 0

    mem = torch.cuda.max_memory_allocated() / 1024**2  
    torch.cuda.reset_peak_memory_stats()

    print(f"\n{name}")
    print(f"Forward:  {fwd:.4f}s")
    if enable_backward:
        print(f"Backward: {bwd:.4f}s")
    print(f"Memory:   {mem:.2f} MB")
   
    return fwd, bwd, mem

In [37]:
import time

def get_batch(dataset, batch_size=2, max_length=16):
    idx = torch.randint(0, len(dataset), (batch_size,)).tolist()
    
    batch = {
        "input_ids": torch.stack([torch.tensor(dataset[i]["input_ids"][:max_length]) for i in idx]).to("cuda"),
        "labels": torch.stack([torch.tensor(dataset[i]["labels"][:max_length]) for i in idx]).to("cuda"),
        "attention_mask": torch.stack([torch.tensor(dataset[i]["attention_mask"][:max_length]) for i in idx]).to("cuda"),
    }
    return batch


def check_time(model, name="", enable_backward=True):
    device = next(model.parameters()).device
    print(f"Model is on device: {device}")

    model.train()
    
    if enable_backward:
        optimizer = torch.optim.AdamW(
            [p for p in model.parameters() if p.requires_grad],
            lr=1e-4
        )
    else:
        optimizer = None

    for _ in range(2):
        batch = get_batch(tokenized_train_dataset, batch_size=1, max_length=32)
        batch = {k: v.to(device) for k, v in batch.items()}
        out = model(**batch)
        loss = out.loss
        if enable_backward:
            optimizer.zero_grad()
            loss.backward()

    torch.cuda.synchronize()

    batch = get_batch(tokenized_train_dataset, batch_size=1, max_length=32)
    batch = {k: v.to(device) for k, v in batch.items()}
    
    start = time.time()
    out = model(**batch)
    torch.cuda.synchronize()
    fwd = time.time() - start

    if enable_backward:
        loss = out.loss
        optimizer.zero_grad()
        start = time.time()
        loss.backward()
        torch.cuda.synchronize()
        bwd = time.time() - start
    else:
        bwd = 0

    mem = torch.cuda.max_memory_allocated() / 1024**2  
    torch.cuda.reset_peak_memory_stats()

    print(f"\n{name}")
    print(f"Forward:  {fwd:.4f}s")
    if enable_backward:
        print(f"Backward: {bwd:.4f}s")
    print(f"Memory:   {mem:.2f} MB")
   
    return fwd, bwd, mem


check_time(model, "Qwen", enable_backward=False)


Model is on device: cuda:0

Qwen
Forward:  0.0396s
Memory:   14646.13 MB


(0.0396420955657959, 0, 14646.13427734375)

In [13]:
check_time(model, "Qwen LoRA", enable_backward=True)

Model is on device: cuda:0

Qwen LoRA
Forward:  0.0593s
Backward: 0.1009s
Memory:   6850.52 MB


(0.05925631523132324, 0.10089516639709473, 6850.5185546875)

С LoRA forward немного дольше, потому что нужно умножить две матрицы A B и потом еще прибавить их к линейному слою.
Для обычного Qwen у меня в кагл не поместилась модель, поэтому backward я не считала (у меня постоянно была out of memory). По памяти модель с лорой занимает почти в два раза меньше места. Генерирует даже после трех эпох неплохо, но постоянно про какую-то Лили, других имен не выбирает, скорее всего особенность датасета, потому что он синтетический тоже  (нагенерен гптшкой)

### Supervised Fine-tuning

__Задание 2 (3 балла).__ Разбейте все примеры с "хорошими" ответами на запросы (все что идет до последнего "Assistant:") и ответы (все, начиная с последнего "Assistant:"). Дообучите модель [`pythia-1.4b`](https://huggingface.co/EleutherAI/pythia-1.4b) генерировать правильные ответы с помощью вашей LoRA. Одной эпохи вполне должно хватить для сходимости. Проверьте на нескольких случайных тестовых примерах, что модель ведет себя так, как надо.

In [5]:
# # your code here
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1.4b")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-1.4b", torch_dtype=torch.bfloat16)

/root/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
  

In [7]:
from datasets import load_dataset

train_dataset = load_dataset("Anthropic/hh-rlhf", split="train")
test_dataset = load_dataset("Anthropic/hh-rlhf", split="test")

In [40]:
train_dataset[1]['chosen']

'\n\nHuman: What kind of noises did dinosaurs make?\n\nAssistant: Humans and dinosaurs didn’t live at the same time, so it’s really hard to say. The best place to find out what noises dinosaurs made would be\n\nHuman: yes they did\n\nAssistant: to guess, and that would probably require lots of reading and a certain amount of imagination, so we’re not really prepared to do that.\n\nHuman: you cant read\n\nAssistant: You can read?'

In [23]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): LinearWithLoRA(
            (linear): Linear(in_features=2048, out_features=6144, bias=True)
            (lora): LoRALayer()
          )
          (dense): LinearWithLoRA(
            (linear): Linear(in_features=2048, out_features=2048, bias=True)
            (lora): LoRALayer()
          )
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): LinearWithLoRA(
            (linear): Linear(in_features=2048, out_features=8192, bia

In [8]:
def clean_dataset(example):
    text = example['chosen'] 
    parts = text.split("Assistant:")
    input_text = "Assistant:".join(parts[:-1])
    target_text = "Assistant:"+ parts[-1] #долго думала добавлять помарку что это ассистент. в итоге решила добавить
    return {"input": input_text, "target": target_text}

train_dataset = train_dataset.map(clean_dataset)
test_dataset = test_dataset.map(clean_dataset)

In [42]:
train_dataset[0]

{'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.",
 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, ji

In [9]:
train_dataset = train_dataset.remove_columns(['chosen', 'rejected']) 
test_dataset = test_dataset.remove_columns(['chosen', 'rejected'])

In [44]:
train_dataset[0]

{'input': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\n",
 'target': "Assistant: I haven't even thought about it."}

In [22]:
tokenizer.pad_token = tokenizer.eos_token

In [10]:
IGNORE_INDEX = -100
max_length = 256
tokenizer.pad_token = tokenizer.eos_token

def tokenize(examples):
    inputs = examples["input"]
    targets = examples["target"]

    all_input_ids = []
    all_attention = []
    all_labels = []

    for inp, tgt in zip(inputs, targets):
        full = inp + tgt

        enc = tokenizer(
            full,
            truncation=True,
            padding="max_length",
            max_length=max_length,
        )

        inp_enc = tokenizer(
            inp,
            truncation=True,
            padding="max_length",
            max_length=max_length,
        )

        input_len = sum(inp_enc["attention_mask"])

        labels = enc["input_ids"].copy()
        labels[:input_len] = [IGNORE_INDEX] * input_len

        all_input_ids.append(enc["input_ids"])
        all_attention.append(enc["attention_mask"])
        all_labels.append(labels)

    return {
        "input_ids": all_input_ids,
        "attention_mask": all_attention,
        "labels": all_labels,
    }

tokenized_train_dataset = train_dataset.map(tokenize, batched=True, remove_columns=train_dataset.column_names)
tokenized_test_dataset = test_dataset.map(tokenize, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/160800 [00:00<?, ? examples/s]

Map: 100%|██████████| 160800/160800 [04:58<00:00, 538.66 examples/s]


In [11]:
for param in model.parameters():
    param.requires_grad = False

model = replace_with_lora(model, rank=8)

In [12]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

Total parameters: 1421358080
Trainable parameters: 6710272


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq

import os

model.config.use_cache = True

import torch
torch.cuda.empty_cache()

model.gradient_checkpointing_disable()

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    padding=True, 
    max_length=max_length,
    label_pad_token_id=IGNORE_INDEX,
)

training_args = TrainingArguments(
    output_dir='./lora_pythia',

    num_train_epochs=1,

    per_device_train_batch_size=16,      
    gradient_accumulation_steps=1,   
    dataloader_num_workers=4,   

    learning_rate=5e-5,
    warmup_ratio=0.03,
    max_steps=5000,

    bf16=True,                         
    fp16=False,

    optim="adamw_torch",

    max_grad_norm=0,                    

    logging_dir='./logs',
    logging_steps=20,
    save_steps=2000,
    evaluation_strategy="no",

    save_safetensors=False,

    dataloader_pin_memory=True,
    remove_unused_columns=True,

    disable_tqdm=False,
    report_to=None,
    logging_first_step=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator
)

trainer.train()


/root/.venv/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just go

Step,Training Loss
1,6.311100
20,6.933400
40,1.892000
60,0.722000
80,0.745000
100,0.774200
120,0.696500
140,0.672500
160,0.694500
180,0.681000


### Direct Preference Optimization

__Задание 3 (3 балла).__ Реализуйте DPO согласно [статье](https://arxiv.org/pdf/2305.18290) и дообучите SFT модель с предыдущего шага. Одной эпохи так же должно хватить, но можно обучать и дольше. Убедитесь, что модель начинает отдавать предпочтение хорошим ответам. Проведите анализ. Стали ли ответы лучше, чем у SFT модели? Всегда ли модель отвечает хорошо или иногда плохо? Насколько легко модель ломается при изменении промптов?